In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, shutil
import pandas as pd
from docx import Document

UserEmail="r.chehab2@exeter.ac.uk"
Password= "Put_Your_Password_Here"
url1 = "https://vle.exeter.ac.uk/mod/assign/view.php?id=2387500&action=grading"

def Open_window(UserEmail, Password, url1):
    EMAILFIELD = (By.ID, "i0116")
    PASSWORDFIELD = (By.ID, "i0118")
    NEXTBUTTON = (By.ID, "idSIButton9")
    Yes = (By.ID, "idSIButton9")

    driver = webdriver.Chrome()
    driver.get(url1)
    # wait for email field and enter email
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(EMAILFIELD)).send_keys(UserEmail)

    # Click Next
    WebDriverWait(driver, 10).until( EC.element_to_be_clickable(NEXTBUTTON)).click()

    # wait for password field and enter password
    WebDriverWait(driver, 10).until( EC.element_to_be_clickable(PASSWORDFIELD)).send_keys(Password)

    # Click Login - same id?
    WebDriverWait(driver, 10).until( EC.element_to_be_clickable(NEXTBUTTON)).click()

    # Make sure you are back on the top of the page
    driver.maximize_window()

    return driver

def get_latest_file():
    # Get the latest downloaded file before new downloads
    download_path = os.path.expanduser('~/Downloads')
    dfiles = os.listdir(download_path)
    previous_file = max(dfiles, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
    src1 = os.path.join(download_path, previous_file)
    return previous_file

def wait_download(previous_file):
    download_path = os.path.expanduser('~/Downloads')
    # Wait for the file to be downloaded by ensuring that the last downloaded file does not match the pervious file
    while True:
        # Get the list of files in the download directory
        dfiles = os.listdir(download_path)

        # Get the last file in the list
        if len(dfiles) > 0:
            last_file = max(dfiles, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
            if previous_file is None or previous_file != last_file:
                break
        time.sleep(1)

def move_download(destination_path):
    download_path = os.path.expanduser('~/Downloads')
    # Get the list of files in the download directory
    dfiles = os.listdir(download_path)

    last_file = max(dfiles, key=lambda x: os.path.getctime(os.path.join(download_path, x)))
    # Move downloaded file into the destination directory
    src = os.path.join(download_path, last_file)
    # dst = os.path.join(destination_path, f)
    shutil.move(src, destination_path)
    print(f"{src} moved to {destination_path}")

def change_name_of_docx():
    download_path = os.path.expanduser('~/Downloads')
    # Get the list of files in the download directory
    dfiles = os.listdir(download_path)
    last_file = max(dfiles, key=lambda x: os.path.getctime(os.path.join(download_path, x)))

    if last_file.endswith('.docx'):
        path1=os.path.join(download_path,last_file)
        doc = Document(path1)
        if len(doc.tables)>0:
            candidate_number=doc.tables[0].cell(1, 1).text
            os.rename(path1, os.path.join(download_path, candidate_number+ '.docx'))
        else: 
            candidate_number='Not defined'
    else:
        candidate_number=''

    return candidate_number

In [2]:
driver1=Open_window(UserEmail, Password, url1)
wait = WebDriverWait(driver1, 40)

In [3]:
# Get the latest file in the downloads
previous_file=get_latest_file()

# Create folder of Students dissertations
folder_name = "Students Dissertations"
os.makedirs(folder_name, exist_ok=True)

# Extract the table using selenium from driver1
participants=[]
candidate_numbers=[]
table = driver1.find_elements(By.CSS_SELECTOR, "table")[0]
rows = table.find_elements(By.TAG_NAME, 'tr')
for row in rows:
    columns = row.find_elements(By.TAG_NAME, 'td')
    if len(columns) > 3:
        if columns[2].text!="No submission\nNot marked":
            participants.append(columns[1].text)
            # Create subfolders in each folder
            path1=os.path.join(folder_name, columns[1].text)
            os.makedirs(path1, exist_ok=True)

            for i in columns[6].find_elements(By.CSS_SELECTOR, "a"):
                href = i.click()
                wait_download(previous_file)
                candidate_numbers.append(change_name_of_docx())
                move_download(path1)

/Users/ramichehab/Downloads/089345.docx moved to Students Dissertations/Participant 1502795
/Users/ramichehab/Downloads/Quantitative Methods essay.docx moved to Students Dissertations/Participant 1502808
/Users/ramichehab/Downloads/250368.docx moved to Students Dissertations/Participant 1502800
/Users/ramichehab/Downloads/Assignment Approval.PNG moved to Students Dissertations/Participant 1502800
/Users/ramichehab/Downloads/212705.docx moved to Students Dissertations/Participant 1502818
/Users/ramichehab/Downloads/BEF2103DA Quantative Essay.pdf moved to Students Dissertations/Participant 1502822
/Users/ramichehab/Downloads/BEF2103DA Quantitative Methods 236723.pdf moved to Students Dissertations/Participant 1502825
/Users/ramichehab/Downloads/162131.docx moved to Students Dissertations/Participant 1502797
/Users/ramichehab/Downloads/VaR as a financial risk management tool.docx moved to Students Dissertations/Participant 1502821
/Users/ramichehab/Downloads/232391.docx moved to Students 